In [0]:
-- Supporting Objects for the ETL Procedure

-- 1. Create logging table for procedure execution tracking
CREATE TABLE IF NOT EXISTS gap_catalog.ads_etl_owner.procedure_log (
    procedure_name STRING,
    load_date STRING,
    process_key STRING,
    execution_time TIMESTAMP,
    status STRING,
    error_message STRING,
    created_at TIMESTAMP DEFAULT current_timestamp()
) USING DELTA
TBLPROPERTIES (
    'delta.feature.allowColumnDefaults' = 'supported',
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
);

-- 2. Create procedure execution wrapper for workflow integration
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_execute_with_logging(
    procedure_name STRING,
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
AS
BEGIN
    DECLARE execution_start TIMESTAMP DEFAULT current_timestamp();
    DECLARE rows_processed INT DEFAULT 0;
    
    -- Log execution start
    INSERT INTO gap_catalog.ads_etl_owner.procedure_log VALUES (
        procedure_name || '_WRAPPER',
        p_load_date,
        p_process_key,
        execution_start,
        'STARTED',
        NULL,
        current_timestamp()
    );
    
    BEGIN
        -- Execute the main procedure
        CALL gap_catalog.ads_etl_owner.sp_ADS_RDS_EVENT_TYPES_MEP(p_load_date, p_process_key);
        
        -- Get row count from target table
        SET rows_processed = (
            SELECT COUNT(*) 
            FROM gap_catalog.ads_owner.EVENT_TYPES 
            WHERE EVETP_INSERT_PROCESS_KEY = CAST(p_process_key AS BIGINT)
        );
        
        -- Log successful completion with metrics
        INSERT INTO gap_catalog.ads_etl_owner.procedure_log VALUES (
            procedure_name || '_WRAPPER',
            p_load_date,
            p_process_key,
            current_timestamp(),
            'COMPLETED',
            CONCAT('Rows processed: ', CAST(rows_processed AS STRING), 
                   ', Duration: ', CAST(TIMESTAMPDIFF(SECOND, execution_start, current_timestamp()) AS STRING), 's'),
            current_timestamp()
        );
        
    EXCEPTION
        WHEN OTHERS THEN
            -- Log error with details
            INSERT INTO gap_catalog.ads_etl_owner.procedure_log VALUES (
                procedure_name || '_WRAPPER',
                p_load_date,
                p_process_key,
                current_timestamp(),
                'FAILED',
                CONCAT('Error after ', CAST(TIMESTAMPDIFF(SECOND, execution_start, current_timestamp()) AS STRING), 's - ', SQLSTATE, ': ', SQLERRM),
                current_timestamp()
            );
            
            -- Re-raise the exception
            SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = CONCAT('Wrapper procedure failed: ', SQLERRM);
    END;
END;

-- 3. Usage Examples

-- Basic procedure call (replace with your actual values)
CALL gap_catalog.ads_etl_owner.sp_ADS_RDS_EVENT_TYPES_MEP(
    '2025-09-24',  -- p_load_date
    '12345'        -- p_process_key
);

-- Call with logging wrapper
CALL gap_catalog.ads_etl_owner.sp_execute_with_logging(
    'sp_ADS_RDS_EVENT_TYPES_MEP',
    '2025-09-24',
    '12345'
);

-- 4. Monitoring Queries

-- Check procedure execution history
SELECT 
    procedure_name,
    load_date,
    process_key,
    execution_time,
    status,
    error_message,
    TIMESTAMPDIFF(SECOND, 
        LAG(execution_time) OVER (PARTITION BY procedure_name, load_date, process_key ORDER BY execution_time), 
        execution_time
    ) as duration_seconds
FROM gap_catalog.ads_etl_owner.procedure_log
WHERE DATE(execution_time) >= current_date() - INTERVAL 7 DAYS
ORDER BY execution_time DESC;

-- Check for failed executions
SELECT *
FROM gap_catalog.ads_etl_owner.procedure_log
WHERE status = 'FAILED'
    AND DATE(execution_time) >= current_date() - INTERVAL 1 DAY
ORDER BY execution_time DESC;

-- Performance monitoring
SELECT 
    procedure_name,
    load_date,
    COUNT(*) as execution_count,
    AVG(TIMESTAMPDIFF(SECOND, 
        LAG(execution_time) OVER (PARTITION BY procedure_name, load_date, process_key ORDER BY execution_time), 
        execution_time
    )) as avg_duration_seconds,
    MAX(TIMESTAMPDIFF(SECOND, 
        LAG(execution_time) OVER (PARTITION BY procedure_name, load_date, process_key ORDER BY execution_time), 
        execution_time
    )) as max_duration_seconds
FROM gap_catalog.ads_etl_owner.procedure_log
WHERE status IN ('COMPLETED', 'STARTED')
    AND DATE(execution_time) >= current_date() - INTERVAL 30 DAYS
GROUP BY procedure_name, load_date
ORDER BY load_date DESC, avg_duration_seconds DESC;

-- 5. Workflow Task Configuration (JSON)
/* 
For your Databricks workflow, replace the current SQL task with:

{
  "task_key": "ADS_RDS_EVENT_TYPES_MEP",
  "sql_task": {
    "warehouse_id": "your-warehouse-id",
    "query": {
      "query": "CALL gap_catalog.ads_etl_owner.sp_execute_with_logging('sp_ADS_RDS_EVENT_TYPES_MEP', '{{p_load_date}}', '{{p_process_key}}')"
    }
  },
  "timeout_seconds": 3600,
  "depends_on": []
}
*/

-- 6. Create view for easy procedure management
CREATE OR REPLACE VIEW gap_catalog.ads_etl_owner.v_procedure_status AS
SELECT 
    procedure_name,
    load_date,
    process_key,
    MAX(CASE WHEN status = 'STARTED' THEN execution_time END) as started_at,
    MAX(CASE WHEN status = 'COMPLETED' THEN execution_time END) as completed_at,
    MAX(CASE WHEN status = 'FAILED' THEN execution_time END) as failed_at,
    CASE 
        WHEN MAX(CASE WHEN status = 'COMPLETED' THEN execution_time END) IS NOT NULL THEN 'COMPLETED'
        WHEN MAX(CASE WHEN status = 'FAILED' THEN execution_time END) IS NOT NULL THEN 'FAILED'
        WHEN MAX(CASE WHEN status = 'STARTED' THEN execution_time END) IS NOT NULL THEN 'RUNNING'
        ELSE 'UNKNOWN'
    END as current_status,
    TIMESTAMPDIFF(SECOND, 
        MAX(CASE WHEN status = 'STARTED' THEN execution_time END),
        COALESCE(
            MAX(CASE WHEN status = 'COMPLETED' THEN execution_time END),
            MAX(CASE WHEN status = 'FAILED' THEN execution_time END)
        )
    ) as duration_seconds
FROM gap_catalog.ads_etl_owner.procedure_log
WHERE DATE(execution_time) >= current_date() - INTERVAL 7 DAYS
GROUP BY procedure_name, load_date, process_key
ORDER BY started_at DESC;

-- Check current procedure status
SELECT * FROM gap_catalog.ads_etl_owner.v_procedure_status
WHERE current_status IN ('RUNNING', 'FAILED')
ORDER BY started_at DESC;



--Patrik Šlechta -- Ada kontakt DBX
--Lukáš Vosecký -- Ada kontakt DBX
--Tomáš Plánička -- Ada kontakt DBX
select * from  gap_catalog.ads_etl_owner.procedure_log ;
 delete from  gap_catalog.ads_etl_owner.procedure_log ;